<a id="main_header"></a>
# Technology and Operations Case NBIM

##### *Written by Petter Stangeland*
___

## Background 
##### NBIM processes ~8,000 dividend events annually across 9,000+ equity holdings, requiring daily reconciliation between NBIM internal booking system and what the global custodian sends us. Manual processes are time-consuming and error-prone. We want to explore how Large Language Models could transform this workflow - from break detection to automated remediation.

## Our Goal
##### Design and implement an LLM-powered system to reconcile the provided dividend data. How could LLM agents improve this process and be a dynamic system identifying issues.


## Data
##### We have two datasets to work with: 
* ##### NBIM_Dividend_Bookings.csv 

    * ##### The NBIM dataset is an internal dividend bookings table per corporate action event, with identifiers (ISIN, SEDOL), key dates (EXDATE, PAYMENT_DATE), amounts (gross/net), currencies/FX, and withholding tax fields.

* ##### CUSTODY_Dividend_Bookings.csv

    * ##### The custody dataset is the custodian’s dividend bookings per event with identifiers, positions and bank account, ex/record/pay dates, gross and net amounts in quotation and settlement currencies, FX, taxes, and ADR or restitution fees.

## Key Questions to Explore
* ##### How can LLMs classify and prioritize reconciliation breaks?
* ##### What types of intelligent agents could automate the entire workflow?
* ##### What safeguards are needed for autonomous financial operations?

## Table of Contents

* ### [Data Exploration](#data_exploration_header)

* ### [Data Wrangling](#data_wrangling_header)

* ### [Break Detection](#break_detection_header)

* ### [Break Classification & Reconciliation](#break_classification_reconciliation_header)

* ### [Agent Architecture](#agent_architecture_header)

* ### [Outlook](#outlook_header)

In [10]:
import pandas as pd
from IPython.display import SVG, display, HTML
from openai import OpenAI
import os
from dotenv import load_dotenv

#### Help functions

In [11]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

def display_side_by_side(dfs:list, captions:list, tablespacing=5):
    """Display tables side by side to save vertical space
    Input:
        dfs: list of pandas.DataFrame
        captions: list of table captions
    """
    output = ""
    for (caption, df) in zip(captions, dfs):
        output += df.style.set_table_attributes("style='display:inline'").set_caption(caption)._repr_html_()
        output += tablespacing * "\xa0"
    display(HTML(output))

load_dotenv()
api_key = os.environ["OPENAI_API_KEY"]
def show_svg():
    #display(SVG('nbimdiagram.svg'))
    svg = SVG(filename="nbimdiagram.svg").data
    display(HTML(f"<div style='text-align:center'>{svg}</div>"))

<a id="data_exploration_header"></a>
## Data Exploration

In [12]:
data_nbim = pd.read_csv("NBIM_Dividend_Bookings 1.csv", sep=";")
data_custody = pd.read_csv("CUSTODY_Dividend_Bookings 1.csv", sep=";")

display_side_by_side([data_nbim, data_custody], ['NBIM', 'Custodian'])

,COAC_EVENT_KEY,INSTRUMENT_DESCRIPTION,ISIN,SEDOL,TICKER,ORGANISATION_NAME,DIVIDENDS_PER_SHARE,EXDATE,PAYMENT_DATE,CUSTODIAN,BANK_ACCOUNT,QUOTATION_CURRENCY,SETTLEMENT_CURRENCY,AVG_FX_RATE_QUOTATION_TO_PORTFOLIO,NOMINAL_BASIS,GROSS_AMOUNT_QUOTATION,NET_AMOUNT_QUOTATION,NET_AMOUNT_SETTLEMENT,GROSS_AMOUNT_PORTFOLIO,NET_AMOUNT_PORTFOLIO,WTHTAX_COST_QUOTATION,WTHTAX_COST_SETTLEMENT,WTHTAX_COST_PORTFOLIO,WTHTAX_RATE,LOCALTAX_COST_QUOTATION,LOCALTAX_COST_SETTLEMENT,TOTAL_TAX_RATE,EXRESPRDIV_COST_QUOTATION,EXRESPRDIV_COST_SETTLEMENT,RESTITUTION_RATE
0,950123456,APPLE INC,US0378331005,2046251,AAPL,Apple Inc,0.250000,07.02.2025,14.02.2025,JPMORGAN_CHASE,501234567,USD,USD,11.234500,1500000,375000,318750,318750.000000,4212937.500000,3580996.880000,56250,56250.000000,631940.630000,15,0,0.000000,15,0,0,0
1,960789012,SAMSUNG ELECTRONICS CO LTD,KR7005930003,6771720,005930 KS,Samsung Electronics Co Ltd,361.000000,31.03.2025,20.05.2025,HSBC_KOREA,712345678,KRW,USD,0.008234,25000,9025000,6769950,5181.500000,74311.850000,55738.630000,1985500,1519.530000,16348.610000,22,269550,206.260000,25,0,0,0
2,970456789,NESTLE SA,CH0038863350,7196907,NESN SW,Nestle SA,3.100000,25.04.2025,29.04.2025,UBS_SWITZERLAND,823456789,CHF,CHF,12.456700,20000,62000,40300,40300.000000,772559.840000,502163.900000,21700,21700.000000,270395.940000,35,0,0.000000,35,0,0,0
3,970456789,NESTLE SA,CH0038863350,7196907,NESN SW,Nestle SA,3.100000,25.04.2025,29.04.2025,UBS_SWITZERLAND,823456790,CHF,CHF,12.456700,15000,46500,30225,30225.000000,579419.880000,376622.920000,16275,16275.000000,202796.960000,35,0,0.000000,35,0,0,0
4,970456789,NESTLE SA,CH0038863350,7196907,NESN SW,Nestle SA,3.100000,25.04.2025,29.04.2025,UBS_SWITZERLAND,823456791,CHF,CHF,12.456700,10000,31000,20150,20150.000000,386279.920000,251081.950000,10850,10850.000000,135197.970000,35,0,0.000000,35,0,0,0
,COAC_EVENT_KEY,ISIN,EVENT_EX_DATE,EVENT_PAYMENT_DATE,CUSTODY,SEDOL,CUSTODIAN,EVENT_TYPE,NOMINAL_BASIS,LOAN_QUANTITY,HOLDING_QUANTITY,LENDING_PERCENTAGE,BANK_ACCOUNTS,EX_DATE,RECORD_DATE,PAY_DATE,CURRENCIES,DIV_RATE,TAX_RATE,GROSS_AMOUNT,NET_AMOUNT_QC,TAX,NET_AMOUNT_SC,SETTLED_CURRENCY,IS_CROSS_CURRENCY_REVERSAL,FX_RATE,POSSIBLE_RESTITUTION_PAYMENT,POSSIBLE_RESTITUTION_AMOUNT,ADR_FEE,ADR_FEE_RATE
0,950123456,US0378331005,07.02.2025,14.02.2025,501234567,2046251,CUST/JPMORGANUS,DVCA,1500000,0,1500000,0,501234567,07.02.2025,08.02.2025,14.02.2025,USD,0.250000,15,375000,318750,56250,318750.000000,USD,False,1.000000,0,0,0,0
1,960789012,KR7005930003,31.03.2025,20.05.2025,712345678,6771720,CUST/HSBCKR,DVCA,25000,2000,23000,8,712345678,31.03.2025,01.04.2025,25.05.2025,KRW USD,361.000000,20,9025000,7220000,1805000,5524.270000,USD,True,1307.250000,0,0,0,0
2,970456789,CH0038863350,25.04.2025,29.04.2025,823456789,7196907,CUST/UBSCH,DVCA,20000,0,20000,0,823456789,25.04.2025,26.04.2025,29.04.2025,CHF,3.100000,35,62000,40300,21700,40300.000000,CHF,False,1.000000,6000,6000,0,0
3,970456789,CH0038863350,25.04.2025,29.04.2025,823456790,7196907,CUST/UBSCH,DVCA,30000,0,15000,0,823456790,25.04.2025,26.04.2025,29.04.2025,CHF,3.100000,35,46500,30225,16275,30225.000000,CHF,False,1.000000,4500,4500,0,0


____

<a id="data_wrangling_header"></a>
## Data Wrangling

##### We need a common langauge between NBIM and the custodian

In [13]:
nbim_to_custody = {"COAC_EVENT_KEY": "COAC_EVENT_KEY", "INSTRUMENT_DESCRIPTION": None, 
             "ISIN": "ISIN", "SEDOL": "SEDOL", "TICKER": None, "ORGANISATION_NAME": None,
             "DIVIDENDS_PER_SHARE": "DIV_RATE", "EXDATE": "EVENT_EX_DATE", "PAYMENT_DATE": "EVENT_PAYMENT_DATE",
             "CUSTODIAN": "CUSTODIAN", "BANK_ACCOUNT": "BANK_ACCOUNTS", "QUOTATION_CURRENCY": "CURRENCIES",
             "SETTLEMENT_CURRENCY": "SETTLED_CURRENCY", "AVG_FX_RATE_QUOTATION_TO_PORTFOLIO": None,
             "NOMINAL_BASIS": "NOMINAL_BASIS", "GROSS_AMOUNT_QUOTATION": "GROSS_AMOUNT",
             "NET_AMOUNT_QUOTATION": "NET_AMOUNT_QC", "NET_AMOUNT_SETTLEMENT": "NET_AMOUNT_SC",
             "GROSS_AMOUNT_PORTFOLIO": None, "NET_AMOUNT_PORTFOLIO": None, "WTHTAX_COST_QUOTATION": "TAX",
             "WTHTAX_COST_SETTLEMENT": None, "WTHTAX_COST_PORTFOLIO": None, "WTHTAX_RATE": "TAX_RATE",
             "LOCALTAX_COST_QUOTATION": None, "LOCALTAX_COST_SETTLEMENT": None, "TOTAL_TAX_RATE": None,
             "EXRESPRDIV_COST_QUOTATION": None, "EXRESPRDIV_COST_SETTLEMENT": None, "RESTITUTION_RATE": None}

custody_to_nbim = {"COAC_EVENT_KEY": "COAC_EVENT_KEY", "ISIN": "ISIN", "EVENT_EX_DATE": "EXDATE",
                   "EVENT_PAYMENT_DATE": "PAYMENT_DATE", "CUSTODY": "BANK_ACCOUNT", "SEDOL": "SEDOL",
                   "CUSTODIAN": "CUSTODIAN", "EVENT_TYPE": None, "NOMINAL_BASIS": "NOMINAL_BASIS",
                   "LOAN_QUANTITY": None, "HOLDING_QUANTITY": None, "LENDING_PERCENTAGE": None,
                   "BANK_ACCOUNTS": "BANK_ACCOUNT", "EX_DATE": None, "RECORD_DATE": None, "PAY_DATE": "PAYMENT_DATE",
                   "CURRENCIES": "QUOTATION_CURRENCY", "DIV_RATE": "DIVIDENDS_PER_SHARE", "TAX_RATE": "WTHTAX_RATE",
                   "GROSS_AMOUNT": "GROSS_AMOUNT_QUOTATION", "NET_AMOUNT_QC": "NET_AMOUNT_QUOTATION", 
                   "TAX": "WTHTAX_COST_QUOTATION","NET_AMOUNT_SC": "NET_AMOUNT_SETTLEMENT", "SETTLED_CURRENCY": "SETTLEMENT_CURRENCY", 
                   "IS_CROSS_CURRENCY_REVERSAL": None, "FX_RATE": None, "POSSIBLE_RESTITUTION_PAYMENT": None,
                   "POSSIBLE_RESTITUTION_AMOUNT": None, "ADR_FEE": None, "ADR_FEE_RATE": None}



# different custodian codes
# different quotation currency codes
# EX_DATE == EVENT_EX_DATE?


custody_cols  = [v for v in nbim_to_custody.values() if v]
nbim_cols = [custody_to_nbim.get(v, "") for v in custody_cols]

# aligned print
w = max(map(len, custody_cols))
for a, b in zip(custody_cols, nbim_cols):
    print(f"{a:<{w}} ---- {b}")


COAC_EVENT_KEY     ---- COAC_EVENT_KEY
ISIN               ---- ISIN
SEDOL              ---- SEDOL
DIV_RATE           ---- DIVIDENDS_PER_SHARE
EVENT_EX_DATE      ---- EXDATE
EVENT_PAYMENT_DATE ---- PAYMENT_DATE
CUSTODIAN          ---- CUSTODIAN
BANK_ACCOUNTS      ---- BANK_ACCOUNT
CURRENCIES         ---- QUOTATION_CURRENCY
SETTLED_CURRENCY   ---- SETTLEMENT_CURRENCY
NOMINAL_BASIS      ---- NOMINAL_BASIS
GROSS_AMOUNT       ---- GROSS_AMOUNT_QUOTATION
NET_AMOUNT_QC      ---- NET_AMOUNT_QUOTATION
NET_AMOUNT_SC      ---- NET_AMOUNT_SETTLEMENT
TAX                ---- WTHTAX_COST_QUOTATION
TAX_RATE           ---- WTHTAX_RATE


##### *Dictionary for words in common between NBIM and the custodian*

##### Cleaned up dataframes are much easier to read and compare

In [14]:
display_side_by_side([data_nbim[nbim_cols], data_custody[custody_cols]], ['NBIM', 'Custodian'])

,COAC_EVENT_KEY,ISIN,SEDOL,DIVIDENDS_PER_SHARE,EXDATE,PAYMENT_DATE,CUSTODIAN,BANK_ACCOUNT,QUOTATION_CURRENCY,SETTLEMENT_CURRENCY,NOMINAL_BASIS,GROSS_AMOUNT_QUOTATION,NET_AMOUNT_QUOTATION,NET_AMOUNT_SETTLEMENT,WTHTAX_COST_QUOTATION,WTHTAX_RATE
0,950123456,US0378331005,2046251,0.250000,07.02.2025,14.02.2025,JPMORGAN_CHASE,501234567,USD,USD,1500000,375000,318750,318750.000000,56250,15
1,960789012,KR7005930003,6771720,361.000000,31.03.2025,20.05.2025,HSBC_KOREA,712345678,KRW,USD,25000,9025000,6769950,5181.500000,1985500,22
2,970456789,CH0038863350,7196907,3.100000,25.04.2025,29.04.2025,UBS_SWITZERLAND,823456789,CHF,CHF,20000,62000,40300,40300.000000,21700,35
3,970456789,CH0038863350,7196907,3.100000,25.04.2025,29.04.2025,UBS_SWITZERLAND,823456790,CHF,CHF,15000,46500,30225,30225.000000,16275,35
4,970456789,CH0038863350,7196907,3.100000,25.04.2025,29.04.2025,UBS_SWITZERLAND,823456791,CHF,CHF,10000,31000,20150,20150.000000,10850,35
,COAC_EVENT_KEY,ISIN,SEDOL,DIV_RATE,EVENT_EX_DATE,EVENT_PAYMENT_DATE,CUSTODIAN,BANK_ACCOUNTS,CURRENCIES,SETTLED_CURRENCY,NOMINAL_BASIS,GROSS_AMOUNT,NET_AMOUNT_QC,NET_AMOUNT_SC,TAX,TAX_RATE
0,950123456,US0378331005,2046251,0.250000,07.02.2025,14.02.2025,CUST/JPMORGANUS,501234567,USD,USD,1500000,375000,318750,318750.000000,56250,15
1,960789012,KR7005930003,6771720,361.000000,31.03.2025,20.05.2025,CUST/HSBCKR,712345678,KRW USD,USD,25000,9025000,7220000,5524.270000,1805000,20
2,970456789,CH0038863350,7196907,3.100000,25.04.2025,29.04.2025,CUST/UBSCH,823456789,CHF,CHF,20000,62000,40300,40300.000000,21700,35
3,970456789,CH0038863350,7196907,3.100000,25.04.2025,29.04.2025,CUST/UBSCH,823456790,CHF,CHF,30000,46500,30225,30225.000000,16275,35


In [15]:
custody_to_nbim_ren = {v:k for k,v in nbim_to_custody.items() if v is not None}

df1 = data_nbim.reindex(columns=nbim_cols)
df2 = data_custody.rename(columns=custody_to_nbim_ren).reindex(columns=nbim_cols)

combined = pd.concat({'NBIM': df1, 'CUSTODY': df2}, axis=1)
combined = combined.swaplevel(0, 1, axis=1).reindex(columns=nbim_cols, level=0)
combined.head()

COAC_EVENT_KEY                     ISIN                  SEDOL          DIVIDENDS_PER_SHARE              EXDATE             PAYMENT_DATE                    CUSTODIAN                  BANK_ACCOUNT            QUOTATION_CURRENCY          SETTLEMENT_CURRENCY         NOMINAL_BASIS          GROSS_AMOUNT_QUOTATION          NET_AMOUNT_QUOTATION          NET_AMOUNT_SETTLEMENT            WTHTAX_COST_QUOTATION          WTHTAX_RATE        
            NBIM    CUSTODY          NBIM       CUSTODY     NBIM  CUSTODY                NBIM CUSTODY        NBIM     CUSTODY         NBIM     CUSTODY             NBIM          CUSTODY         NBIM    CUSTODY               NBIM  CUSTODY                NBIM CUSTODY          NBIM  CUSTODY                   NBIM  CUSTODY                 NBIM  CUSTODY                  NBIM    CUSTODY                  NBIM  CUSTODY        NBIM CUSTODY
0      950123456  950123456  US0378331005  US0378331005  2046251  2046251                0.25    0.25  07.02.2025  07.02.2025   14.02.2025  14.02.2025   JPMORGAN_CHASE  CUST/JPMORGANUS    501234567  501234567                USD      USD                 USD     USD       1500000  1500000                 375000   375000               318750   318750              318750.0  318750.00                 56250    56250          15      15
1      960789012  960789012  KR7005930003  KR7005930003  6771720  6771720              361.00  361.00  31.03.2025  31.03.2025   20.05.2025  20.05.2025       HSBC_KOREA      CUST/HSBCKR    712345678  712345678                KRW  KRW USD                 USD     USD         25000    25000                9025000  9025000              6769950  7220000                5181.5    5524.27               1985500  1805000          22      20
2      970456789  970456789  CH0038863350  CH0038863350  7196907  7196907                3.10    3.10  25.04.2025  25.04.2025   29.04.2025  29.04.2025  UBS_SWITZERLAND       CUST/UBSCH    823456789  823456789                CHF      CHF                 CHF     CHF         20000    20000                  62000    62000                40300    40300               40300.0   40300.00                 21700    21700          35      35
3      970456789  970456789  CH0038863350  CH0038863350  7196907  7196907                3.10    3.10  25.04.2025  25.04.2025   29.04.2025  29.04.2025  UBS_SWITZERLAND       CUST/UBSCH    823456790  823456790                CHF      CHF                 CHF     CHF         15000    30000                  46500    46500                30225    30225               30225.0   30225.00                 16275    16275          35      35
4      970456789  970456789  CH0038863350  CH0038863350  7196907  7196907                3.10    3.10  25.04.2025  25.04.2025   29.04.2025  29.04.2025  UBS_SWITZERLAND       CUST/UBSCH    823456791  823456791                CHF      CHF                 CHF     CHF         10000    10000                  31000    37200                20150    24180               20150.0   24180.00                 10850    13020          35      35

##### *Even easier to read and compare once we combine into one dataframe*
____

<a id="break_detection_header"></a>
## Break Detection

In [16]:
client = OpenAI(api_key=api_key)


csv = combined.to_csv(index=False)
model = "gpt-5-2025-08-07"
prompt = prompt = (
    "For each column in the csv compare the values in the two sub-columns NBIM and CUSTODY. Find all places where they differ." + csv
)

resp = client.responses.create(
    model=model,
    input=prompt
)

In [17]:
print(resp.output_text)

Here are all NBIM vs CUSTODY mismatches by row.

Row 1 (COAC_EVENT_KEY=950123456, ISIN=US0378331005):
- CUSTODIAN: NBIM=JPMORGAN_CHASE, CUSTODY=CUST/JPMORGANUS

Row 2 (COAC_EVENT_KEY=960789012, ISIN=KR7005930003):
- CUSTODIAN: NBIM=HSBC_KOREA, CUSTODY=CUST/HSBCKR
- QUOTATION_CURRENCY: NBIM=KRW, CUSTODY=KRW USD
- NET_AMOUNT_QUOTATION: NBIM=6769950, CUSTODY=7220000
- NET_AMOUNT_SETTLEMENT: NBIM=5181.5, CUSTODY=5524.27
- WTHTAX_COST_QUOTATION: NBIM=1985500, CUSTODY=1805000
- WTHTAX_RATE: NBIM=22, CUSTODY=20

Row 3 (COAC_EVENT_KEY=970456789, ISIN=CH0038863350, BANK_ACCOUNT=823456789):
- CUSTODIAN: NBIM=UBS_SWITZERLAND, CUSTODY=CUST/UBSCH

Row 4 (COAC_EVENT_KEY=970456789, ISIN=CH0038863350, BANK_ACCOUNT=823456790):
- CUSTODIAN: NBIM=UBS_SWITZERLAND, CUSTODY=CUST/UBSCH
- NOMINAL_BASIS: NBIM=15000, CUSTODY=30000

Row 5 (COAC_EVENT_KEY=970456789, ISIN=CH0038863350, BANK_ACCOUNT=823456791):
- CUSTODIAN: NBIM=UBS_SWITZERLAND, CUSTODY=CUST/UBSCH
- GROSS_AMOUNT_QUOTATION: NBIM=31000, CUSTODY=37200

____

<a id="break_classification_reconciliation_header"></a>
## Break Classification & Reconciliation

##### *Reconciliation breaks*
* ##### **Problem 1:** Notice the recurring issue that CUSTODY uses CUSTODIAN format: CUST/BANKNAMECOUNTRYCODE with country codes US-United States, KR-Korea, CH-Switzerland, while NBIM uses more irregular format: JPMORGAN_CHASE, HSBC_KOREA, UBS_SWITZERLAND. <br> <br> **FIX:** Easy fix just convert between them with synonym dictionary

* ##### **Problem 2:** Another recurring issue is the currency format, CUSTODY: FRSTCURR SCNDCURR, NBIM: CURR. With the limited amount of documentation for the data can not say if CUSTODY only does this if the quotation currency is different from the settlement currency, but this seems to be the case. <br> <br> **FIX:** Easy fix as well, just convert between them with synonym dictionary

* ##### **Problem 3:** Different values in columns NET_AMOUNT_QUOTATION, NET_AMOUNT_SETTLEMENT, WTHTAX_COST_QUOTATION, WTHTAX_RATE, NOMINAL_BASIS, GROSS_AMOUNT_QUOTATION <br> <br>**FIX:** More difficult to fix. Possible solutions: If NBIM data is usually correct in the case where they differ, continue with the NBIM data, if CUSTODY data is uaully correct in the case where they differ, continue with CUSTODY data. On the other hand if it is not possible to decide which one is correct or it is random which one is correct when they differ, continue with average between them. Additionally we can use alternative data from Bloomberg or similar

##### *Reconciliation break prioritization*
##### ChatGPT suggestions:

##### Simple scoring model (lower = higher priority):

##### Base weight by class:
1. ##### missing value 
2. ##### currency/FX
3. ##### amount 
4. ##### tax 
5. ##### date issue 
6. ##### rounding

##### Add penalties:
* ##### +0 if |amount| ≥ high_threshold else +1 (mid) else +2 (low)
* ##### +0 if days_to_pay ≤ 1 else +1 (≤3) else +2 (>3)
* ##### −1 if root-cause confidence ≥ 0.9 else 0

##### Add aging bonus: +min(days_open//2, 3)

##### *Reconciliation break classification*

In [18]:
client = OpenAI(api_key=api_key)


csv = combined.to_csv(index=False)
model = "gpt-5-2025-08-07"
prompt = prompt = (
    "For each COAC_EVENT_KEY int the csv classify what error has occurred such that the NBIM data differs from the CUSTODY data." + csv
)

resp = client.responses.create(
    model=model,
    input=prompt
)

In [19]:
print(resp.output_text)

Here’s a concise classification of the NBIM vs CUSTODY differences by COAC_EVENT_KEY (broken out by bank account where the key appears multiple times):

- 950123456
  - Custodian alias mismatch: NBIM “JPMORGAN_CHASE” vs CUSTODY “CUST/JPMORGANUS”.
  - All dates, quantities, currencies, and amounts match.

- 960789012
  - Withholding tax rate mismatch: NBIM 22% vs CUSTODY 20% → causes differences in WTHTAX_COST_QUOTATION, NET_AMOUNT_QUOTATION, and NET_AMOUNT_SETTLEMENT.
  - Quotation currency field malformed on CUSTODY: “KRW USD” instead of “KRW”.
  - Custodian alias mismatch: NBIM “HSBC_KOREA” vs CUSTODY “CUST/HSBCKR”.

- 970456789
  - Bank account 823456789:
    - Custodian alias mismatch: NBIM “UBS_SWITZERLAND” vs CUSTODY “CUST/UBSCH”.
    - Amounts, quantities, and rates match.
  - Bank account 823456790:
    - Nominal basis mismatch: NBIM 15,000 vs CUSTODY 30,000, while gross/net amounts reflect 15,000 → CUSTODY nominal is incorrect.
  - Bank account 823456791:
    - Amounts oversta

____

<a id="agent_architecture_header"></a>
## Agent Architecture

In [20]:
show_svg()

____

<a id="outlook_header"></a>
## Outlook

##### Here are three examples of possible use cases of this technology

* ##### KYC/Onboarding - Julius Bär

* ##### ESG Monitoring over a enormous stock universe - NBIM

* ##### FX Regime classification - NBIM

##### However, the biggest take-away, for me at least, is to not restrict ourselves to just these use cases, but rather to recognize how immensely powerful and flexible AI agents truly are

##### Of course, *with great power comes greate responsibility*

##### Mitigation and safeguard strategies:

* ##### Transparent prompts 

* ##### Additional deterministic checks

* ##### Human supervision

* ##### read-only until approved

[back to top](#main_header)